In [1]:
import pandas as pd
import numpy as np
import sys
import torch
import torch.nn as nn

In [2]:
df_train = pd.read_csv(r"C:\Users\maxim\Desktop\js\linfit\test\archive\mnist_train.csv")
df_test = pd.read_csv(r"C:\Users\maxim\Desktop\js\linfit\test\archive\mnist_test.csv")

In [3]:
xcolnams = df_train.columns.tolist()
print("check0")
df_train[xcolnams[1:]] /= 255. #convert the pixel values to 0..1
print("check1")

Xs_train = torch.tensor(df_train[xcolnams[1:]].values).float().cuda()
print("check2")

ycolnams = []
for i in range(10):
    coln = "is"+str(i)
    ycolnams.append(coln)
    df_train[coln] = (df_train["label"] == i).astype(float)

check0
check1
check2


In [4]:
xcolnams = df_test.columns.tolist()
print("check0")
df_test[xcolnams[1:]] /= 255. #convert the pixel values to 0..1
print("check1")

Xs_test = torch.tensor(df_test[xcolnams[1:]].values).float().cuda()
print("check2")

ycolnams = []
for i in range(10):
    coln = "is"+str(i)
    ycolnams.append(coln)
    df_test[coln] = (df_test["label"] == i).astype(float)

check0
check1
check2


In [5]:
Ys_train = torch.tensor(df_train[ycolnams].values).float().cuda()
Ys_test = torch.tensor(df_test[ycolnams].values).float().cuda()

In [6]:
print(Xs_test.shape)
print(Ys_train[:,:10])

torch.Size([10000, 784])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]], device='cuda:0')


In [171]:
mod = nn.Sequential(
    nn.Linear(28**2, 400),
    nn.ReLU(),
    nn.Linear(400, 10),
    nn.ReLU()
)

# mod = nn.Sequential(
#     nn.Linear(28**2, 10),
# )

mod = mod.cuda()

In [172]:
Xs_train.shape

torch.Size([60000, 784])

In [173]:
mod(Xs_train)

tensor([[0.0000, 0.0687, 0.1029,  ..., 0.0191, 0.0535, 0.0000],
        [0.0000, 0.0955, 0.0000,  ..., 0.0726, 0.0000, 0.0000],
        [0.0000, 0.0925, 0.0000,  ..., 0.0285, 0.0000, 0.0000],
        ...,
        [0.0000, 0.1100, 0.1026,  ..., 0.0315, 0.0850, 0.0000],
        [0.0000, 0.0480, 0.0191,  ..., 0.0446, 0.0000, 0.0000],
        [0.0000, 0.1393, 0.0290,  ..., 0.0264, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)

In [174]:
import linstorch

In [175]:
import importlib
importlib.reload(linstorch)

<module 'linstorch' from 'c:\\Users\\maxim\\Desktop\\js\\linfit\\fulltorch\\linstorch.py'>

In [176]:
# t0 = time.time()
# mod = linstorch.solvemodel(mod, Xs_train, Ys_train)
# print('{} seconds'.format(time.time() - t0))

In [177]:
forw = mod.forward(Xs_train)
guessed = torch.argmax(forw, axis=1).cpu().numpy()
print(sum(guessed == df_train["label"].values) / len(guessed))

0.07565


In [178]:
forw = mod.forward(Xs_test)
guessed = torch.argmax(forw, axis=1).cpu().numpy()
print(sum(guessed == df_test["label"].values) / len(guessed))

0.0792


In [179]:
trainloader=torch.utils.data.DataLoader(list(zip(Xs_train, Ys_train)), batch_size=32, shuffle=True)
testloader=torch.utils.data.DataLoader(list(zip(Xs_test, Ys_test)), batch_size=32, shuffle=False)

In [180]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(mod.parameters(), lr=0.001)
#optimizer = optim.SGD(mod.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adagrad(mod.parameters(), lr=0.001)

def train(ep):
    for epoch in range(ep):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            # print(labels.shape)
            labels = torch.argmax(labels, axis=1)
            # print(labels)
            # print(labels.shape)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = mod(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')

In [181]:
import time

In [182]:
t0 = time.time()
train(1)
print('{} seconds'.format(time.time() - t0))

Finished Training
2.24399995803833 seconds


In [183]:
forw = mod.forward(Xs_test)
guessed = torch.argmax(forw, axis=1).cpu().numpy()
print(sum(guessed == df_test["label"].values) / len(guessed))

0.785
